In [1]:
import pandas as pd

In [2]:
df = pd.read_json(path_or_buf="case.json")

In [3]:
df.sample(n=12, random_state=55)

,EnqueuedTimeUtc,EventName,Payload
5,2021-08-18 11:44:00 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPricePerOption"",""offe..."
18,2021-09-05 09:03:52 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
23,2021-09-05 09:03:41 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
28,2021-09-05 09:02:21 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
4,2021-09-05 08:03:28 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
25,2021-09-05 09:02:05 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
9,2021-09-05 09:18:04 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
6,2021-09-05 08:03:40 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
33,2021-09-05 09:02:41 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
20,2021-08-18 11:43:57 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPricePerOption"",""offe..."


In [4]:
df.shape

(37, 3)

In [5]:
df["EnqueuedTimeUtc"] = pd.to_datetime(arg=df["EnqueuedTimeUtc"]).dt.tz_convert(tz="Brazil/East")

In [6]:
df["EventName"].value_counts()


EventName
DynamicPrice_Result    32
CurateOffer_Result      5
Name: count, dtype: int64

In [7]:
dynamic_price_events = df[df["EventName"] == "DynamicPrice_Result"]
dynamic_price_events.shape

(32, 3)

In [8]:
dynamic_price_events.head()

,EnqueuedTimeUtc,EventName,Payload
0,2021-09-05 05:04:08-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
1,2021-08-18 08:43:23-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPricePerOption"",""offe..."
2,2021-09-05 06:04:04-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
4,2021-09-05 05:03:28-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
5,2021-08-18 08:44:00-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPricePerOption"",""offe..."


In [9]:
import json
import pandas as pd

def dynamic_price_option(row):
    """Transform the JSON payload into a dynamic price option dataframe."""
    enqueued_time = row["EnqueuedTimeUtc"].strftime("%d/%m/%y")
    payload_row = json.loads(row["Payload"])
    
    if payload_row["provider"] != "ApplyDynamicPricePerOption":
        return []
    
    provider = payload_row["provider"]
    offer_id = payload_row["offerId"]
    
    return [
        {
            "Provider": provider,
            "OfferId": offer_id,
            "UniqueOptionId": option["uniqueOptionId"],
            "BestPrice": option["bestPrice"],
            "EnqueuedTimeSP": enqueued_time
        }
        for option in payload_row["algorithmOutput"]
    ]

# Apply the function and flatten the list of lists into a single DataFrame
dynamic_price_option_payload = pd.DataFrame(
    [option for row in dynamic_price_events.apply(dynamic_price_option, axis=1) for option in row]
)

In [10]:
dynamic_price_option_payload

,Provider,OfferId,UniqueOptionId,BestPrice,EnqueuedTimeSP
0,ApplyDynamicPricePerOption,56e0702c-0218-4626-8d3d-ae9d54b4503b,b0e296a9-0590-f0e0-8211-243a2ededb12,92.45,18/08/21
1,ApplyDynamicPricePerOption,56e0702c-0218-4626-8d3d-ae9d54b4503b,d6562c24-0b37-5fb4-8275-65b7b8b47b87,92.45,18/08/21
2,ApplyDynamicPricePerOption,56e0702c-0218-4626-8d3d-ae9d54b4503b,8d0f9262-f543-d0c8-a869-33985ae3ecda,92.45,18/08/21
3,ApplyDynamicPricePerOption,56e0702c-0218-4626-8d3d-ae9d54b4503b,151e59ac-761a-96f5-d2b9-882037a9fd28,94.60,18/08/21
4,ApplyDynamicPricePerOption,56e0702c-0218-4626-8d3d-ae9d54b4503b,3cd346f4-d297-7568-2e50-d43a8e2fd0a9,94.60,18/08/21
5,ApplyDynamicPricePerOption,56e0702c-0218-4626-8d3d-ae9d54b4503b,b7a7b6d1-4dae-7392-5aaf-f3369c29db1d,93.00,18/08/21
6,ApplyDynamicPricePerOption,56e0702c-0218-4626-8d3d-ae9d54b4503b,577e4bbd-f49d-ac23-56a6-e70072a05229,93.00,18/08/21
7,ApplyDynamicPricePerOption,56e0702c-0218-4626-8d3d-ae9d54b4503b,f9b876ab-2590-952f-d69d-5b352ec251f3,91.35,18/08/21
8,ApplyDynamicPricePerOption,00991873-194e-4a6e-89c9-8f68668b6aaa,b0e296a9-0590-f0e0-8211-243a2ededb12,92.45,18/08/21
9,ApplyDynamicPricePerOption,00991873-194e-4a6e-89c9-8f68668b6aaa,d6562c24-0b37-5fb4-8275-65b7b8b47b87,92.45,18/08/21


In [11]:
dynamic_price_option_payload.to_csv("dynamic_price_option_payload.csv", index=False)

In [12]:
import json
import pandas as pd

def dynamic_price_range(row):
    """Transform the JSON payload into a dynamic price range dataframe."""
    enqueued_time = row["EnqueuedTimeUtc"].strftime("%d/%m/%y")
    payload_row = json.loads(row["Payload"])
    
    if payload_row["provider"] != "ApplyDynamicPriceRange":
        return []
    
    provider = payload_row["provider"]
    offer_id = payload_row["offerId"]
    algorithm_output = payload_row["algorithmOutput"]
    
    price_range = {
        "Provider": provider,
        "OfferId": offer_id,
        "MinGlobal": algorithm_output["min_global"],
        "MinRecommended": algorithm_output["min_recommended"],
        "MaxRecommended": algorithm_output["max_recommended"],
        "DifferenceMinRecommendMinTheory": algorithm_output["differenceMinRecommendMinTheory"],
        "EnqueuedTimeSP": enqueued_time
    }
    
    return [price_range]

# Apply the function and flatten the list of lists into a single DataFrame
dynamic_price_range_payload = pd.DataFrame(
    [price_range for row in dynamic_price_events.apply(dynamic_price_range, axis=1) for price_range in row]
)
